## AI Travel Agent with Microsoft Agent Framework

In [1]:
! pip install agent-framework-core -U

  Attempting uninstall: agent-framework-core
    Found existing installation: agent-framework-core 1.0.0b251016
    Uninstalling agent-framework-core-1.0.0b251016:
      Successfully uninstalled agent-framework-core-1.0.0b251016


In [4]:
# 📦 Import Required Libraries
# Standard library imports for system operations and random number generation
import os
from random import randint

# Third-party library for loading environment variables from .env file
from dotenv import load_dotenv

In [5]:
# 🤖 Import Microsoft Agent Framework Components
# ChatAgent: The main agent class for conversational AI
# OpenAIChatClient: Client for connecting to OpenAI-compatible APIs (including GitHub Models)
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [6]:
# 🔧 Load Environment Variables
# This loads configuration from a .env file in the project root
# Required variables: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
load_dotenv()

True

In [7]:
# 🎲 Tool Function: Random Destination Generator
# This function will be available to the agent as a tool
# The agent can call this function to get random vacation destinations
def get_random_destination() -> str:
    """Get a random vacation destination.
    
    Returns:
        str: A randomly selected destination from our predefined list
    """
    # List of popular vacation destinations around the world
    destinations = [
        "Barcelona, Spain",
        "Paris, France", 
        "Berlin, Germany",
        "Tokyo, Japan",
        "Sydney, Australia",
        "New York, USA",
        "Cairo, Egypt",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bali, Indonesia"
    ]
    # Return a random destination from the list
    return destinations[randint(0, len(destinations) - 1)]

In [48]:
# 🔗 Create OpenAI Chat Client for GitHub Models
# This client connects to GitHub Models API (OpenAI-compatible endpoint)
# Environment variables required:
# - GITHUB_ENDPOINT: API endpoint URL (usually https://models.inference.ai.azure.com)
# - GITHUB_TOKEN: Your GitHub personal access token
# - GITHUB_MODEL_ID: Model to use (e.g., gpt-4o-mini, gpt-4o)
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"),
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id=os.environ.get("GITHUB_MODEL_ID")
)

In [9]:
# 🤖 Create the Travel Planning Agent
# This creates a conversational AI agent with specific capabilities:
# - chat_client: The AI model client for generating responses
# - instructions: System prompt that defines the agent's personality and role
# - tools: List of functions the agent can call to perform actions
agent = ChatAgent(
    chat_client=openai_chat_client,
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations.",
    tools=[get_random_destination]  # Our random destination tool function
)

In [10]:
# 🚀 Run the Agent
# Send a message to the agent and get a response
# The agent will use its tools (get_random_destination) if needed
response = await agent.run("Plan me a day trip")

In [11]:
# 📋 View Raw Response Object
# This shows the complete response structure including metadata
# Useful for debugging and understanding the response format
response

In [12]:
# 📖 Extract and Display the Travel Plan
# Get the last message from the conversation (agent's response)s
last_message = response.messages[-1]
# Extract the text content from the message
text_content = last_message.contents[0].text
# Display the formatted travel plan
print("🏖️ Travel plan:")
print(text_content)

🏖️ Travel plan:
How about a day trip to Tokyo, Japan? Here's a suggested itinerary for your day:

### Morning:
- **Breakfast:** Start your day with a traditional Japanese breakfast. Try a local café for a rice porridge (okayu) or miso soup and grilled fish.
- **Visit Senso-ji Temple:** Head to Asakusa to visit Tokyo’s oldest temple. Enjoy the bustling atmosphere of Nakamise Street leading up to the temple where you can find traditional snacks and souvenirs.

### Afternoon:
- **Lunch in Akihabara:** Explore the electronics district and enjoy a meal at a themed café (like a maid café) for a unique experience.
- **Akihabara Exploration:** Check out the stores for anime, manga, and gaming merchandise. You can also visit stores showcasing cutting-edge technology.

### Late Afternoon:
- **Shibuya Crossing:** Make your way to Shibuya and witness the iconic Shibuya Crossing, one of the busiest pedestrian crossings in the world. Take photos and enjoy the view.
- **Shopping in Shibuya:** Explore

### Agentic Design Patterns

In [32]:
from random import randint 

In [33]:
# 🛠️ Tool Function Design Pattern
# Implements the Strategy Pattern for pluggable agent capabilities
# This demonstrates clean separation of business logic from agent orchestration
def get_random_destination() -> str:
    """Get a random vacation destination using Repository Pattern.
    
    This function exemplifies several design patterns:
    - Strategy Pattern: Interchangeable algorithm for destination selection
    - Repository Pattern: Encapsulates data access logic
    - Factory Method: Creates destination objects on demand
    
    Returns:
        str: A randomly selected destination following consistent format
    """
    # Data Repository Pattern: Centralized destination data management
    destinations = [
        "Barcelona, Spain",      # Mediterranean cultural hub
        "Paris, France",         # European artistic center
        "Berlin, Germany",       # Historical European capital
        "Tokyo, Japan",          # Asian technology metropolis
        "Sydney, Australia",     # Oceanic coastal city
        "New York, USA",         # American urban center
        "Cairo, Egypt",          # African historical capital
        "Cape Town, South Africa", # African scenic destination
        "Rio de Janeiro, Brazil",  # South American beach city
        "Bali, Indonesia"          # Southeast Asian island paradise
    ]
    
    # Factory Method Pattern: Create destination selection on demand
    return destinations[randint(0, len(destinations) - 1)]

In [36]:
AGENT_NAME ="TravelAgent"

AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers.

Important: When users specify a destination, always plan for that location. Only suggest random destinations when the user hasn't specified a preference.

When the conversation begins, introduce yourself with this message:
"Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. Here are some things you can ask me:
1. Plan a day trip to a specific location
2. Suggest a random vacation destination
3. Find destinations with specific features (beaches, mountains, historical sites, etc.)
4. Plan an alternative trip if you don't like my first suggestion

What kind of trip would you like me to help you plan today?"

Always prioritize user preferences. If they mention a specific destination like "Bali" or "Paris," focus your planning on that location rather than suggesting alternatives.
"""

In [37]:
agent = ChatAgent(
        name = AGENT_NAME,
        chat_client=openai_chat_client,
        instructions=AGENT_INSTRUCTIONS,
        tools=[get_random_destination]
)

In [38]:
thread = agent.get_new_thread()

In [39]:
response1 = await agent.run("Plan me a day trip",thread= thread)

In [40]:
last_message = response1.messages[-1]
text_content = last_message.contents[0].text
print("Travel plan:")
print(text_content)

Travel plan:
Hello! I'm your TravelAgent assistant. I can help plan vacations and suggest interesting destinations for you. 

To get started, could you please specify a location for your day trip?


In [41]:
response2 = await agent.run("I don't like that destination. Plan me another vacation.",thread= thread)

In [42]:
last_message = response2.messages[-1]
text_content = last_message.contents[0].text
print("Change plan:")
print(text_content)

Change plan:
I understand! However, I need you to specify a destination or let me know if you would like me to suggest a random vacation destination for you. Please let me know your preference!


### Use 🛠️ Advanced Tool 

In [46]:
# � Import core dependencies for Agent Framework and tool integration
# This sets up the essential libraries for building intelligent agents with tool capabilities

import asyncio
import os
import json

from dotenv import load_dotenv  # For loading environment variables securely
from random import randint

# These are the core components for building tool-enabled agents
from agent_framework import ChatAgent           # Main agent class
from agent_framework.openai import OpenAIChatClient  # OpenAI-compatible client

In [47]:
# 🛠️ Define travel planning tools for agent integration
# These functions provide specific capabilities that the agent can invoke dynamically

def get_random_destination() -> str:
    """
    🎲 Random destination generator tool
    Returns a randomly selected travel destination from curated list
    Useful when customers need inspiration for their next vacation
    """
    destinations = [
        "Paris, France",
        "Tokyo, Japan", 
        "New York City, USA",
        "London, England",
        "Rome, Italy",
        "Sydney, Australia",
        "Dubai, UAE",
        "Barcelona, Spain",
        "Bangkok, Thailand",
        "Amsterdam, Netherlands",
        "Istanbul, Turkey",
        "Prague, Czech Republic",
        "Santorini, Greece",
        "Reykjavik, Iceland",
        "Marrakech, Morocco",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bali, Indonesia"
    ]
    # 🎯 Return random selection from the curated destination list
    return destinations[randint(0, len(destinations) - 1)]

In [49]:
# 🤖 Configure travel agent identity and behavioral instructions
# Define the agent's personality, capabilities, and operational guidelines

AGENT_NAME = "TravelAgent"

AGENT_INSTRUCTIONS = """You are a helpful AI Agent that can help plan vacations for customers at random destinations
"""

In [50]:
agent = ChatAgent(
        name = AGENT_NAME,
        chat_client=openai_chat_client,
        instructions=AGENT_INSTRUCTIONS,
        tools=[get_random_destination]
)

In [51]:
thread = agent.get_new_thread()

In [52]:
response1 = await agent.run("Plan me a day trip",thread= thread)

In [ ]:
last_message = response1.messages[-1]
text_content = last_message.contents[0].text
print("Travel plan:")
print(text_content)

Travel plan:
How about a day trip to Marrakech, Morocco? Here’s a suggested itinerary for your adventure:

### Morning:
- **Breakfast in a Local Café**: Start your day with traditional Moroccan pastries and mint tea.
- **Visit the Jardin Majorelle**: Stroll through this beautiful garden filled with exotic plants and the vibrant blue colors of the buildings.

### Midday:
- **Explore the Medina**: Wander through the bustling streets of the old city. Don't forget to visit the famous souks where you can shop for spices, textiles, and pottery.
- **Lunch at a Local Restaurant**: Try a traditional Moroccan dish like tagine or couscous at a cafe with a view of the bustling market.

### Afternoon:
- **Koutoubia Mosque**: Take photos of the stunning mosque, which is a landmark of Marrakech (note that non-Muslims cannot enter).
- **Visit the Saadian Tombs**: Discover the intricate carvings and architecture of these historic tombs.

### Evening:
- **Djemaa el-Fna Square**: Head to the heart of Mar

: 

### Agentic RAG